In [2]:
import pandas as pd
df = pd.read_csv("retail_sales_dataset.csv")
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,24-11-2023,CUST001,Male,34,Beauty,3,50,150
1,2,27-02-2023,CUST002,Female,26,Clothing,2,500,1000
2,3,13-01-2023,CUST003,Male,50,Electronics,1,30,30
3,4,21-05-2023,CUST004,Male,37,Clothing,1,500,500
4,5,06-05-2023,CUST005,Male,30,Beauty,2,50,100


In [3]:
print("Shape:", df.shape)
df.info()
df.head()

Shape: (1000, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    1000 non-null   int64 
 1   Date              1000 non-null   object
 2   Customer ID       1000 non-null   object
 3   Gender            1000 non-null   object
 4   Age               1000 non-null   int64 
 5   Product Category  1000 non-null   object
 6   Quantity          1000 non-null   int64 
 7   Price per Unit    1000 non-null   int64 
 8   Total Amount      1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,24-11-2023,CUST001,Male,34,Beauty,3,50,150
1,2,27-02-2023,CUST002,Female,26,Clothing,2,500,1000
2,3,13-01-2023,CUST003,Male,50,Electronics,1,30,30
3,4,21-05-2023,CUST004,Male,37,Clothing,1,500,500
4,5,06-05-2023,CUST005,Male,30,Beauty,2,50,100


In [4]:
df.isna().sum()

,0
Transaction ID,0
Date,0
Customer ID,0
Gender,0
Age,0
Product Category,0
Quantity,0
Price per Unit,0
Total Amount,0


In [6]:
df.duplicated().sum()

np.int64(0)

In [7]:
rows_before = df.shape[0]

df = df.drop_duplicates()

rows_after = df.shape[0]

print("Rows before removing duplicates:", rows_before)
print("Rows after removing duplicates:", rows_after)

Rows before removing duplicates: 1000
Rows after removing duplicates: 1000


In [8]:
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

cat_cols = df.select_dtypes(include=["object"]).columns
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

In [9]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)
print(df.columns)

Index(['transaction_id', 'date', 'customer_id', 'gender', 'age',
       'product_category', 'quantity', 'price_per_unit', 'total_amount'],
      dtype='object')


In [10]:
df["date"] = pd.to_datetime(df["date"], dayfirst=True)
numeric_cols = ["age", "quantity", "price_per_unit", "total_amount"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    1000 non-null   int64         
 1   date              1000 non-null   datetime64[ns]
 2   customer_id       1000 non-null   object        
 3   gender            1000 non-null   object        
 4   age               1000 non-null   int64         
 5   product_category  1000 non-null   object        
 6   quantity          1000 non-null   int64         
 7   price_per_unit    1000 non-null   int64         
 8   total_amount      1000 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 70.4+ KB


In [11]:
df["calculated_total"] = df["quantity"] * df["price_per_unit"]

In [12]:
threshold = df["total_amount"].median()
df["high_value_order"] = df["total_amount"] > threshold

In [13]:
df["order_year"] = df["date"].dt.year
df["order_month"] = df["date"].dt.month

In [14]:
def age_group(age):
    if age < 25:
        return "young"
    elif age < 40:
        return "adult"
    else:
        return "senior"

df["age_group"] = df["age"].apply(age_group)

In [15]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    1000 non-null   int64         
 1   date              1000 non-null   datetime64[ns]
 2   customer_id       1000 non-null   object        
 3   gender            1000 non-null   object        
 4   age               1000 non-null   int64         
 5   product_category  1000 non-null   object        
 6   quantity          1000 non-null   int64         
 7   price_per_unit    1000 non-null   int64         
 8   total_amount      1000 non-null   int64         
 9   calculated_total  1000 non-null   int64         
 10  high_value_order  1000 non-null   bool          
 11  order_year        1000 non-null   int32         
 12  order_month       1000 non-null   int32         
 13  age_group         1000 non-null   object        
dtypes: bool(1), datetime64[ns

In [16]:
customers = df[[
    "customer_id", "gender", "age", "age_group"
]].drop_duplicates().reset_index(drop=True)

print("Customers shape:", customers.shape)
customers.head()

Customers shape: (1000, 4)


,customer_id,gender,age,age_group
0,CUST001,Male,34,adult
1,CUST002,Female,26,adult
2,CUST003,Male,50,senior
3,CUST004,Male,37,adult
4,CUST005,Male,30,adult


In [17]:
orders = df[[
    "transaction_id", "date", "customer_id",
    "quantity", "price_per_unit", "total_amount",
    "calculated_total", "high_value_order",
    "order_year", "order_month"
]]

print("Orders shape:", orders.shape)
orders.head()

Orders shape: (1000, 10)


,transaction_id,date,customer_id,quantity,price_per_unit,total_amount,calculated_total,high_value_order,order_year,order_month
0,1,2023-11-24,CUST001,3,50,150,150,True,2023,11
1,2,2023-02-27,CUST002,2,500,1000,1000,True,2023,2
2,3,2023-01-13,CUST003,1,30,30,30,False,2023,1
3,4,2023-05-21,CUST004,1,500,500,500,True,2023,5
4,5,2023-05-06,CUST005,2,50,100,100,False,2023,5


In [18]:
products = (
    df.groupby("product_category")
      .agg(
          total_quantity=("quantity", "sum"),
          avg_price=("price_per_unit", "mean"),
          total_sales=("total_amount", "sum")
      )
      .reset_index()
)

print("Products shape:", products.shape)
products.head()

Products shape: (3, 4)


,product_category,total_quantity,avg_price,total_sales
0,Beauty,771,184.055375,143515
1,Clothing,894,174.287749,155580
2,Electronics,849,181.900585,156905


In [19]:
import os
os.makedirs("processed", exist_ok=True)
os.makedirs("output", exist_ok=True)

customers.to_csv("output/customers.csv", index=False)
orders.to_csv("output/orders.csv", index=False)
products.to_csv("output/products.csv", index=False)

print("CSV files saved in output/ folder")

CSV files saved in output/ folder


In [20]:
import sqlite3
conn = sqlite3.connect("database.sqlite")
customers.to_sql("customers", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)

print("Tables saved to database.sqlite")
conn.close()

Tables saved to database.sqlite


In [21]:
conn = sqlite3.connect("database.sqlite")

print(pd.read_sql("SELECT COUNT(*) FROM customers", conn))
print(pd.read_sql("SELECT COUNT(*) FROM orders", conn))
print(pd.read_sql("SELECT COUNT(*) FROM products", conn))

conn.close()

   COUNT(*)
0      1000
   COUNT(*)
0      1000
   COUNT(*)
0         3


In [23]:
print("=== VALIDATION SUMMARY ===")

print("Raw rows (before cleaning):", 1000)
print("Rows after cleaning:", df.shape[0])

print("Orders rows:", orders.shape[0])
print("Unique customers:", customers.shape[0])
print("Products categories:", products.shape[0])

print("\nCheck totals:")
print("Orders rows == Raw rows ?", orders.shape[0] == df.shape[0])
print("Customers <= Orders ?", customers.shape[0] <= orders.shape[0])
print("Products categories =", products.shape[0])

=== VALIDATION SUMMARY ===
Raw rows (before cleaning): 1000
Rows after cleaning: 1000
Orders rows: 1000
Unique customers: 1000
Products categories: 3

Check totals:
Orders rows == Raw rows ? True
Customers <= Orders ? True
Products categories = 3


# Validation Results:
*Raw dataset rows: 1000
*Rows after cleaning: 1000
*Orders table rows: 1000
*Customers table rows: 1000 (unique customers)
*Products table rows: 3 (Beauty, Clothing, Electronics)
*No data loss during transformation
*Row counts match expectations

In [27]:
readme_text = """
# Retail Sales ETL Pipeline

## Project Overview
This project implements an ETL (Extract, Transform, Load) pipeline using a retail sales dataset.

The pipeline:
- Extracts data from a CSV file
- Cleans and transforms the data
- Creates derived features
- Splits data into analytical tables
- Loads data into SQLite and CSV files

## Steps
1. Load raw dataset
2. Clean missing values and duplicates
3. Standardize columns and datatypes
4. Create derived columns (calculated_total, high_value_order, order_year, order_month, age_group)
5. Split into customers, orders, products tables
6. Export to CSV and load into SQLite
7. Validate row counts before and after transformation

## Validation Results
- Raw dataset rows: 1000
- Rows after cleaning: 1000
- Orders table rows: 1000
- Customers table rows: 1000 (unique customers)
- Products table rows: 3 (Beauty, Clothing, Electronics)
- No data loss during transformation
- Row counts match expectations

## Outputs
- task14_etl.ipynb
- output/customers.csv
- output/orders.csv
- output/products.csv
- database.sqlite
"""

with open("README.md", "w") as f:
    f.write(readme_text)

print("README.md file created!")

README.md file created!
